## Simple user interface to demonstrate the project

In [1]:
import os
import cv2
from pathlib import Path

from tkinter import *
import tkinter as tk
from tkinter import font, filedialog, messagebox

import numpy as np
import PIL
from PIL import Image, ImageTk, ImageOps

import keras
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array

import matplotlib.pyplot as plt
from matplotlib import image as mpimg

In [2]:
# DCT

# load img as arr
def load_img(img):
    return cv2.imread(img)

# convert rgb to yub
def rgb2yuv(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

# Split channel
def split_channel(img):
    return cv2.split(img)

# Get scale alpha
def get_Q(q_jpeg):
    QTable = np.array([[16,11,10,16, 24,40, 51, 61],
                   [12,12,14,19, 26,48, 60, 55],
                   [14,13,16,24, 40,57, 69, 56],
                   [14,17,22,29, 51,87, 80, 62],
                   [18,22,37,56, 68,109,103,77],
                   [24,35,55,64, 81,104,113,92],
                   [49,64,78,87,103,121,120,101],
                   [72,92,95,98,112,100,103,99]])
    if q_jpeg < 50 and q_jpeg > 1:
        alpha = np.floor(5000.0/q_jpeg)
        Q = QTable*alpha
    elif q_jpeg < 100:
        alpha = np.floor(200 - 2*q_jpeg)
        Q = QTable*alpha
    elif q_jpeg == 100:
        Q = np.ones((8, 8))
    else:
        raise AttributeError("Quality Factor must be in the range [1..100]")
    return Q

def get_mse(prediction, target):
    return np.sqrt(((prediction-target)**2).mean())

def get_psnr(mse):
    return 20*np.log10(255/np.sqrt(mse))

#######
def demo(qf, source, dimension_info):
    QF = qf
    # Init block 8x8
    block = 8
    h, w = np.array(source.shape[:2])/block*block
    h, w = int(h), int(w)
    blocksV = int(h/block)
    blocksH = int(w/block)
    vis0 = np.zeros((h, w), np.float32)
    result = np.zeros((h, w), np.float32)
    vis0[:h, :w] = source
    Q = get_Q(QF)

    # Perform DCT 8x8
    for row in range(blocksV):
        for col in range(blocksH):
            current_block = vis0[row*block:(row+1)*block, col*block:(col+1)*block]
            dct = cv2.dct(current_block)
            result_ = np.round(dct/Q).astype(int)
            result[row*block:(row+1)*block, col*block:(col+1)*block] = result_

    # Convert back use IDCT
    back = np.zeros((h, w), np.float32)
    for row in range(blocksV):
        for col in range(blocksH):
            current_block = result[row*block:(row+1)*block, col*block:(col+1)*block]
            idct = cv2.idct(current_block*Q)
            back[row*block:(row+1)*block, col*block:(col+1)*block] = np.round(idct).astype(int)

    global y
    mse = get_mse(back, y)
    psnr = np.round(get_psnr(mse), 2)

    # For 1/4 dimension
    if(dimension_info == 4):
        r=0
        c=0
        new_result_arr = np.zeros((blocksV*2, blocksH*2), np.float32)
        temp =0
        for row in range(blocksV*block):
            for col in range(blocksH*block):
                if((row%8 == 0) and (col%8 == 0) ):
                    new_result_arr[r][c] = result[row][col]
                    new_result_arr[r][c+1] = result[row][col+1]
                    new_result_arr[r+1][c] = result[row+1][col]
                    new_result_arr[r+1][c+1] = result[row+1][col+1]
                    c = c+2
            temp = temp+1
            if(temp %8==0):
                r = r+2
            c=0
        new_result_arr = new_result_arr.astype(np.uint8)
        img  = Image.fromarray(new_result_arr)
        return img

    # For 1/8 dimension
    elif(dimension_info == 8):
        r=0
        c=0
        new_result_arr = np.zeros((blocksV, blocksH), np.float32)
        temp =0
        for row in range(blocksV*block):
            for col in range(blocksH*block):
                if((row%8 == 0) and (col%8 == 0) ):
                    new_result_arr[r][c] = result[row][col]
                    c = c+1
            temp = temp+1
            if(temp %8==0):
                r = r+1
            c=0
        new_result_arr = new_result_arr.astype(np.uint8)
        img  = Image.fromarray(new_result_arr)
        return img

    # For same dimension
    else: 
        back = back.astype(np.uint8)
        img  = Image.fromarray(back)
        return img

In [11]:
# Tkinter

if not os.path.exists('tmp'):
    os.makedirs('tmp')

def upload_original_img():
    global img, original_img, original_path
    global original_dim_h, original_dim_w, original_KB
    original_path = filedialog.askopenfilename(initialdir="C:\\Users\ESRA\Desktop\Image-Classification-with-JPG-Data\source\test_demo", title = "Load Image", filetypes=[('Images', ['*jpeg','*png','*jpg', '*tif'])])

    if original_path:
        img = Image.open(original_path)
        original_dim_h, original_dim_w = img.size
        original_KB= os.stat(original_path)
        original_KB = round (((original_KB.st_size) / 1024), 3) # find KB of an image
        img = img.resize((400,400), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        original_img.configure(image = img)

def upload_dct_img():
    global img2, dct_img, original_path, y
    global dct_h, dct_w, dct_KB
    # Load source.jpg
    source = load_img(original_path)
    print(original_path)
    # convert source.jpg to YUV Channel
    yuv = rgb2yuv(source)
    # split YUV channel
    y, u, v = split_channel(yuv)
    # Perform DCT with Y Channel as source
    np.set_printoptions(precision=4, suppress=True)
    source = y
    qf2 = 93.0
    back = demo(qf2, source, 0)
    back.save('tmp\\dct.jpg')
    dct_h, dct_w = back.size
    dct_KB =os.stat('tmp\\dct.jpg')
    dct_KB = round (((dct_KB.st_size) / 1024), 3) # find KB of an image

    img2 = back.resize((300,300), Image.ANTIALIAS)
    img2 = ImageTk.PhotoImage(img2)
    dct_img.configure(image = img2)

def upload_dct_img_dimension4():
    global img2, dct_img, original_path, y
    global dct_dim4_h, dct_dim4_w, dct_dim4_KB
    # Load source.jpg
    source = load_img(original_path)
    print(original_path)
    # convert source.jpg to YUV Channel
    yuv = rgb2yuv(source)
    # split YUV channel
    y, u, v = split_channel(yuv)
    # Perform DCT with Y Channel as source
    np.set_printoptions(precision=4, suppress=True)
    source = y
    qf = 100.0
    back = demo(qf, source, 4)

    back.save('tmp\\dct_dimension4.jpg')
    dct_dim4_h, dct_dim4_w = back.size
    dct_dim4_KB =os.stat('tmp\\dct_dimension4.jpg')
    dct_dim4_KB = round (((dct_dim4_KB.st_size) / 1024), 3) # find KB of an image

    img2 = back.resize((220,220), Image.ANTIALIAS)
    img2 = ImageTk.PhotoImage(img2)
    
    dct_img = Label(root, text = "DCT Image", borderwidth=5, relief="sunken")
    dct_img.place(x = 550,y = 110, height=280, width=350)
    dct_img.configure(image = img2)


def upload_dct_img_dimension8():
    global img2, dct_img, original_path, y
    global dct_dim8_h, dct_dim8_w, dct_dim8_KB
    # Load source.jpg
    source = load_img(original_path)
    print(original_path)
    # convert source.jpg to YUV Channel
    yuv = rgb2yuv(source)
    # split YUV channel
    y, u, v = split_channel(yuv)
    # Perform DCT with Y Channel as source
    np.set_printoptions(precision=4, suppress=True)
    source = y
    qf = 100.0
    back = demo(qf, source, 8)

    back.save('tmp\\dct_dimension8.jpg')
    dct_dim8_h, dct_dim8_w = back.size
    dct_dim8_KB =os.stat('tmp\\dct_dimension8.jpg')
    dct_dim8_KB = round (((dct_dim8_KB.st_size) / 1024), 3) # find KB of an image

    img2 = back.resize((120,120), Image.ANTIALIAS)
    img2 = ImageTk.PhotoImage(img2)
    
    dct_img = Label(root, text = "DCT Image", borderwidth=5, relief="sunken")
    dct_img.place(x = 550,y = 110, height=280, width=350)
    dct_img.configure(image = img2)


def classify_result():
    global prediction
    # Recreate the exact same model, including weights and optimizer
    reconstructed_model = keras.models.load_model("C:\\Users\ESRA\Desktop\Image-Classification-with-JPG-Data\source\HDF5_files\classification_with_dct.h5")
    image = mpimg.imread(original_path)
    plt.imshow(image)
    plt.show()
    image = keras.utils.load_img(original_path, target_size = (64, 64))
    image = img_to_array(image)
    image = np.expand_dims(image, axis = 0)
    result = reconstructed_model.predict(image)

    if result[0][0] == 1:
        prediction_res = 'dog'
    else:
        prediction_res = 'cat'
    print(prediction_res)
    prediction["text"]=prediction_res


def upload_infos():
    original = "-Original Image-" + "\n" + str(original_dim_h)  + "x"+ str(original_dim_w) + "\n" + str(original_KB) + " KB" + "\n\n"
    dct = "-DCT Image-" + "\n" + str(dct_h)  + "x"+ str(dct_w) + "\n" + str(dct_KB) + " KB" + "\n\n"
    dct_4 = "-DCT Image (dim/4)-" + "\n" + str(dct_dim4_h)  + "x"+ str(dct_dim4_w) + "\n" + str(dct_dim4_KB) + " KB" + "\n\n"
    dct_8 = "-DCT Image (dim/8)-" + "\n" + str(dct_dim8_h)  + "x"+ str(dct_dim8_w) + "\n" + str(dct_dim8_KB) + " KB" + "\n"
    infos["text"] = original + dct + dct_4 + dct_8


global y
global prediction
global original_dim_h, original_dim_w, original_KB
global dct_h, dct_w, dct_KB
global dct_dim4_h, dct_dim4_w, dct_dim4_KB
global dct_dim8_h, dct_dim8_w, dct_dim8_KB

#initialise GUI
root=tk.Tk()
root.geometry('1220x620')
root.title('GTU Graduation Project')
root.configure(background="azure4")

heading = Label(root, text="~  Image Classification with JPG Data  ~",pady=5, font=('georgia',20,'bold'))
heading.configure(background='azure4',foreground='red4')
heading.pack()

buttonFont = font.Font(family='Helvetica', size=16, weight='bold')
btn_upload_img = Button(root, text = "Upload Image",bd = 8, fg = "black", bg = "plum4", command = upload_original_img, font=buttonFont)
btn_upload_img.place(x = 150, y = 60, height=40, width=210)

global original_path
global original_img
original_img = Label(root, text = "Original Image", borderwidth=5, relief="sunken")
original_img.place(x = 20,y = 110, height=430, width=500)

global dct_img    # dct image sample with same dimension

btn_upload_dct = Button(root, text = "DCT",bd = 8, fg = "black", bg = "plum4", command = upload_dct_img, font=buttonFont)
btn_upload_dct.place(x = 530, y = 60, height=40, width=130)

dct_img = Label(root, text = "DCT Image Sample", borderwidth=5, relief="sunken")
dct_img.place(x = 550,y = 110, height=280, width=350)

#
btn_upload_dct_4 = Button(root, text = "DCT (dim/4)",bd = 8, fg = "black", bg = "plum4", command = upload_dct_img_dimension4, font=buttonFont)
btn_upload_dct_4.place(x = 665, y = 60, height=40, width=130)
#
btn_upload_dct_8 = Button(root, text = "DCT (dim/8)",bd = 8, fg = "black", bg = "plum4", command = upload_dct_img_dimension8, font=buttonFont)
btn_upload_dct_8.place(x = 800, y = 60, height=40, width=130)

global infos
infoFont = font.Font(family='Consolas', size=13, weight='bold')
btn_infos = Button(root, text = "INFO",bd = 8, fg = "black", bg = "plum4", command = upload_infos, font=buttonFont)
btn_infos.place(x = 1050, y = 60, height=40, width=80)

infos = Label(root, text = "Infos", borderwidth=5, relief="groove", font=infoFont)
infos.place(x = 1000,y = 110, height=400, width=200)

btn_classify = Button(root, text = "CLASSIFY", bg = "plum4",bd = 8, fg = "black", command = classify_result, font = buttonFont)
btn_classify.place(x = 570,y = 460, height=110, width=170)

resultFont = font.Font(family='Helvetica', size=40, weight='bold')
prediction = Label(root, text = "?", borderwidth=5, relief="solid", font=resultFont)
prediction.place(x = 770,y = 450, height=130, width=130)


root.mainloop()


C:/Users/ESRA/Desktop/Image-Classification-with-JPG-Data/source/test_demo/cat1.jpg
C:/Users/ESRA/Desktop/Image-Classification-with-JPG-Data/source/test_demo/cat1.jpg
C:/Users/ESRA/Desktop/Image-Classification-with-JPG-Data/source/test_demo/cat1.jpg


# test

In [2]:
import time

# Recreate the exact same model, including weights and optimizer
reconstructed_model = keras.models.load_model("C:\\Users\ESRA\Desktop\Image-Classification-with-JPG-Data\source\HDF5_files\classification_with_dct.h5")

def test_1000_imgs_cat():
    test_folder_cats = "test_1000_imgs\\cats_500"
                        # C:\Users\ESRA\Desktop\Image-Classification-with-JPG-Data\source\test_1000_imgs\cats_500
    # temp for prediction comparison
    temp = 0
    for demo_image in os.listdir(test_folder_cats):
        tempPath = os.path.join(test_folder_cats, demo_image)
        image = mpimg.imread(tempPath)
        image = keras.utils.load_img(tempPath, target_size = (64, 64))
        image = img_to_array(image)
        image = np.expand_dims(image, axis = 0)
        result = reconstructed_model.predict(image)
        if result[0][0] == 1:
            prediction = 'dog'
        else:
            prediction = 'cat'
            temp = temp + 1
    return temp


def test_1000_imgs_dog():
    test_folder_dogs = "test_1000_imgs\\dogs_500"
                        # C:\\Users\ESRA\Desktop\Image-Classification-with-JPG-Data\source\test_1000_imgs\dogs_500
    temp = 0
    for demo_image in os.listdir(test_folder_dogs):
        tempPath = os.path.join(test_folder_dogs, demo_image)
        image = mpimg.imread(tempPath)
        image = keras.utils.load_img(tempPath, target_size = (64, 64))
        image = img_to_array(image)
        image = np.expand_dims(image, axis = 0)
        result = reconstructed_model.predict(image)
        if result[0][0] == 1:
            prediction = 'dog'
            temp = temp + 1
        else:
            prediction = 'cat'
    return temp


start = time.time()
cat = test_1000_imgs_cat()
dog = test_1000_imgs_dog()
total_time = time.time() - start

print("(Cat prediction result out of 500 cat images) temp : ", cat)
print("(Dog prediction result out of 500 dog images) temp : ", dog)

print("\n(DCT classification) Total time: ", total_time, "seconds")

1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 22ms/step
(Cat prediction result out of 500 cat images) temp :  391
(Dog prediction result out of 500 dog images) temp :  138

(DCT classification) Total time:  79.71461391448975 seconds


In [3]:
import time

# Recreate the exact same model, including weights and optimizer
reconstructed_model = keras.models.load_model("C:\\Users\ESRA\Desktop\Image-Classification-with-JPG-Data\source\HDF5_files\classification_without_dct.h5")

def test_1000_imgs_cat():
    test_folder_cats = "test_1000_imgs\\cats_500"
    # temp for prediction comparison
    temp = 0
    for demo_image in os.listdir(test_folder_cats):
        tempPath = os.path.join(test_folder_cats, demo_image)
        image = mpimg.imread(tempPath)
        image = keras.utils.load_img(tempPath, target_size = (64, 64))
        image = img_to_array(image)
        image = np.expand_dims(image, axis = 0)
        result = reconstructed_model.predict(image)
        if result[0][0] == 1:
            prediction = 'dog'
        else:
            prediction = 'cat'
            temp = temp + 1

    return temp

def test_1000_imgs_dog():
    test_folder_dogs = "test_1000_imgs\\dogs_500"
    temp = 0
    for demo_image in os.listdir(test_folder_dogs):
        tempPath = os.path.join(test_folder_dogs, demo_image)
        image = mpimg.imread(tempPath)
        image = keras.utils.load_img(tempPath, target_size = (64, 64))
        image = img_to_array(image)
        image = np.expand_dims(image, axis = 0)
        result = reconstructed_model.predict(image)
        if result[0][0] == 1:
            prediction = 'dog'
            temp = temp + 1
        else:
            prediction = 'cat'

    return temp

start = time.time()
cat = test_1000_imgs_cat()
dog = test_1000_imgs_dog()
total_time = time.time() - start

print("(Cat prediction result out of 500 cat images) temp : ", cat)
print("(Dog prediction result out of 500 dog images) temp : ", dog)
print("\n(Normal classification) Total time : ", total_time, "seconds")

1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 30ms/step
(Cat prediction result out of 500 cat images) temp :  199
(Dog prediction result out of 500 dog images) temp :  484

(Normal classification) Total time :  84.32389068603516 seconds
